In [1]:
import numpy as np
import pandas as pd
data=pd.read_csv("products.csv",sep=",", encoding='latin-1')
data = data.dropna()
df = pd.DataFrame(data, columns=['Rate', 'Summary'])
df['Rate'] = df['Rate'].apply(lambda x: 0 if not x.isnumeric() else int(x))

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Summary'])
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(df['Summary'])
max_sequence_length = max([len(seq) for seq in sequences])
X = pad_sequences(sequences, maxlen=max_sequence_length)

In [5]:
y = np.array(df['Rate'].astype(int))

X = X[:10000,:]
y = y[:10000]

In [6]:
#Preparing data review
X.shape

(10000, 108)

In [24]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(vocab_size, 200, input_length=max_sequence_length))
model.add(LSTM(512))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

# Evaluating the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

Epoch 1/10
 70/125 [===============>..............] - ETA: 1:16 - loss: -601.5834 - accuracy: 0.1011

KeyboardInterrupt: 

In [9]:
model.save('resources/lstm2.h5')

In [10]:
from keras.models import load_model
model = load_model('resources/lstm2.h5')

In [21]:
summary = [['good'],['Very bad product it\'s a only a fan']]


In [22]:

new_sequence = tokenizer.texts_to_sequences(summary)

new = pad_sequences(new_sequence, maxlen=108)
tests = np.array(new)

In [23]:
predictions = model.predict(tests)
for text, prediction in zip(new_texts, predictions):
    sentiment = 'Positive' if prediction >= 0.5 else 'Negative'
    print("Summary:", text[0])
    print("Sentiment:", sentiment)


1/1 [==============================] - 0s 26ms/step
Summary: UNSATISFACTION
Sentiment: Positive
Summary: VERY BAD QUALITY
Sentiment: Positive


In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# import numpy as np

# encoder = OneHotEncoder(sparse_output=False)

# X = encoder.fit_transform(X)
# X = X.reshape(-1, 1)


In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# import numpy as np
# import re
# v = TfidfVectorizer()
# X_flat = X.ravel()
# X_tfidf = v.fit_transform(X_flat)
# y_int = y.astype(int)


###### 